In [6]:
"""
PROYECTO PROE
Este código se utiliza para calibrar la cámara del sistema de visión, para ejecutarlo se debe tener instalada la librería
openCV y tener al menos 10 imágenes de calibración en la misma carpeta. 
"""
"!pip install opencv-python"

import cv2
import cv2 as cv
import numpy as np
import os
import glob

# Define las dimensiones del del tablero
CHECKERBOARD = (6,9) #Número filas de vértices por número de columnas de vértices
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 0.001) #cada cuadrado del patrón mide 40 mm

# Guarda los puntos 3D del tablero
objpoints = []
# Guarda los puntos 2D del tablero
imgpoints = [] 


# Define las coordenadas globales para los puntos en 3D
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

#Dirección de las imagénes de calibración
images = glob.glob('./Imagenes_Calibracion_Nueva_Sala/*.jpg') 
for fname in images: 
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Encuentra las esquinas del patrón de calibración
    # Cuando se encuentran las esquinas deseadas ret = True
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    Si se detecta el número deseado de esquinas,
    refinamos las coordenadas de píxeles y mostramos
    ellos en las imágenes del tablero de ajedrez
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        
        imgpoints.append(corners2)

        # Dibuja los puntos en las esquinas
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    
    cv2.imshow('img',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

h,w = img.shape[:2]

"""
Realización de la calibración de la cámara 
pasando el valor de puntos 3D conocidos (objpoints)
y las correspondientes coordenadas de píxeles de las
esquinas detectadas (imgpoints)
"""

#Obtiene la matriz de calibración, traslación y rotación
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


#Ejecutar si se desea verificar la eliminación de la distorsión en una imagen de prueba, se debe de
#tener la imagen de prueba en el mismo folder y llamarla "TestImage.jpg"


#Se elimina la distorsión en una imagen para verificar
img = cv.imread('2024-02-12_13-17-57.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

#Elimina distorsión
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# recorta la imagen
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst) #Guarda una imagen sin distorsión


# Se guardan las matrices necesarias para la calibración
np.savetxt("dist.txt", dist)
np.savetxt("cameraMatrix.txt", mtx)

#Verifica la exactitud de los valores encontrados
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "Total error: {}".format(mean_error/len(objpoints)) )


#Imprime las matrices de calibración
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)


Total error: 0.03562433431481449
Camera matrix : 

[[6.55183495e+03 0.00000000e+00 5.46169280e+02]
 [0.00000000e+00 6.34867950e+03 5.46540193e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[ 1.99857512e+00  6.86169319e+01 -5.70528813e-02  5.41013898e-03
  -2.65117039e+03]]
rvecs : 

(array([[ 0.37269015],
       [-0.03851295],
       [-3.02601041]]), array([[ 0.30457854],
       [-0.14256243],
       [-2.48555114]]), array([[ 0.24441494],
       [-0.25879408],
       [-1.49392702]]), array([[ 0.34544216],
       [ 0.02874257],
       [-3.02652834]]), array([[-0.14111792],
       [-0.24468619],
       [ 1.56212972]]), array([[-0.03280986],
       [-0.21411963],
       [ 0.00785636]]), array([[-0.22036882],
       [-0.24661986],
       [ 1.58502827]]), array([[-0.06867313],
       [-0.26518438],
       [ 0.09828111]]), array([[-0.28409591],
       [ 0.29257318],
       [-1.36578708]]), array([[ 0.30772445],
       [-0.1470091 ],
       [-2.45191957]]))
tvecs : 

(array([